In [1]:
pip install pandas

     |████████████████████████████████| 11.7 MB 19.5 MB/s            
     |████████████████████████████████| 16.8 MB 4.3 MB/s             
You should consider upgrading via the '/opt/conda/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import findspark
findspark.init()

In [3]:
from pyspark import SparkConf
from pyspark.sql import SparkSession

In [2]:
import os

In [3]:
ip = os.getenv("MY_POD_IP")

'10.233.105.15'

In [6]:
config = {
    "spark.executor.instances": "1",
    "spark.executor.memory": "1024mb",
    "spark.executor.cores": "1",
    
    "spark.driver.port": "2222",
    "spark.driver.host": ip,
    
    "spark.driver.blockManager.port": "7777",
    "spark.driver.bindAddress": "0.0.0.0",
    "spark.files.overwrite": True,
    
    "spark.kubernetes.namespace": "default",
#     "spark.kubernetes.authenticate.caCertFile": "./ca.crt",
#     "spark.kubernetes.authenticate.oauthTokenFile": "./token",
    "spark.kubernetes.container.image": "datamechanics/spark:3.2.0-hadoop-3.3.1-java-11-scala-2.12-python-3.8-latest",
    
    "spark.hadoop.fs.s3a.access.key": "6DKBB4FU8HIDEJD49GCF",
    "spark.hadoop.fs.s3a.secret.key": "QtELTrZUy402hQ5wCUowfUaiVpgWS821FaU4iFP6",
    "spark.hadoop.fs.s3a.endpoint": "ewr1.vultrobjects.com",
    
    "spark.hadoop.fs.s3a.path.style.access": "true",
    # "spark.hadoop.fs.s3a.connection.ssl.enabled": "false",
    "spark.hadoop.fs.s3a.impl": "org.apache.hadoop.fs.s3a.S3AFileSystem",
    
    # "spark.hadoop.com.amazonaws.services.s3.enableV4": "true",
}

In [7]:
def get_spark_session(app_name: str, config: dict, enable_hive=True):
    
    conf = SparkConf()
    conf.setMaster("k8s://https://kubernetes.default.svc.cluster.local")
    
    for key, value in config.items():
        conf.set(key, value)    
    
    # prepare spark sesseion to be returned
    spark = SparkSession.builder

    spark = spark.appName(app_name)
    spark = spark.enableHiveSupport() if enable_hive else spark


    return spark.config(conf=conf).getOrCreate() 
     

In [8]:
spark = get_spark_session("getting_started", config)

In [14]:
url = "https://my.api.mockaroo.com/users.json?key=c16156b0"

In [15]:
from urllib.request import urlopen
jsonData = urlopen(url).read().decode('utf-8')

In [16]:
rdd = spark.sparkContext.parallelize([jsonData])
df = spark.read.json(rdd)
df.createOrReplaceTempView("users")

In [17]:
df.printSchema()
df.show()

root
 |-- email: string (nullable = true)
 |-- first_name: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- id: long (nullable = true)
 |-- ip_address: string (nullable = true)
 |-- last_name: string (nullable = true)

+--------------------+----------+-----------+---+---------------+----------+
|               email|first_name|     gender| id|     ip_address| last_name|
+--------------------+----------+-----------+---+---------------+----------+
|apennigar0@abouta...|      Abbi|       Male|  1|136.161.110.205|  Pennigar|
|      maust1@ftc.gov|     Margo|     Female|  2|186.113.154.243|      Aust|
|mlosano2@chicagot...|     Marcy|     Female|  3|165.211.128.180|    Losano|
|    bkayne3@webs.com|  Benjamen|     Female|  4|  41.96.213.210|     Kayne|
|kblackaller4@goog...|   Kamilah|       Male|  5| 151.212.54.152|Blackaller|
|lhumfrey5@rediff.com|      Land| Non-binary|  6|  158.7.168.215|   Humfrey|
|  pfawlks6@yandex.ru|     Perla|       Male|  7|  49.196.100.45|    

In [18]:
df2 = spark.sql("SELECT * FROM users as u1 full join users as u2 ")

In [19]:
df2.count()

1000000

In [ ]:
# spark.stop()